In [ ]:
# ATTENTION: only run this cell when on google colab
!git clone https://github.com/philshiu/Drosophila_brain_model.git
!pip install brian2
%cd Drosophila_brain_model

In [1]:
import brian2cuda
brian2cuda.example_run()

/root/anaconda3/envs/brain/lib/python3.10/site-packages/setuptools/_distutils/_msvccompiler.py:12: UserWarning: _get_vc_env is private; find an alternative (pypa/distutils#340)
  warnings.warn(
INFO       CUDA installation directory detected via location of `nvcc` binary: /usr [brian2cuda.utils.gputools]
INFO       Compiling device code for GPU 0 (NVIDIA GeForce RTX 4090 D) [brian2cuda.utils.gputools]
INFO       Compiling device code for compute capability 8.9 (compiler flags: ['-arch=sm_89']) [brian2cuda.device]
INFO       Using the following preferences for CUDA standalone: [brian2cuda.device]
INFO       	devices.cuda_standalone.SM_multiplier = 1 [brian2cuda.device]
INFO       	devices.cuda_standalone.parallel_blocks = 1 [brian2cuda.device]
INFO       	devices.cuda_standalone.launch_bounds = False [brian2cuda.device]
INFO       	devices.cuda_standalone.syn_launch_bounds = False [brian2cuda.device]
INFO       	devices.cuda_standalone.calc_occupancy = True [brian2cuda.device]
INFO     

INFO: _init_arrays() took 0.050994s
INFO _run_kernel_neurongroup_group_variable_set_conditional_codeobject
	1 blocks
	768 threads
	10 registers per thread
	0 bytes statically-allocated shared memory per block
	0 bytes local memory per thread
	720 bytes user-allocated constant memory
	1.000 theoretical occupancy
INFO _run_kernel_neurongroup_stateupdater_codeobject
	1 blocks
	768 threads
	33 registers per thread
	0 bytes statically-allocated shared memory per block
	0 bytes local memory per thread
	720 bytes user-allocated constant memory
	1.000 theoretical occupancy
INFO _run_kernel_neurongroup_spike_thresholder_codeobject
	1 blocks
	768 threads
	18 registers per thread
	0 bytes statically-allocated shared memory per block
	0 bytes local memory per thread
	720 bytes user-allocated constant memory
	1.000 theoretical occupancy
INFO _run_kernel_neurongroup_spike_resetter_codeobject
	1 blocks
	768 threads
	24 registers per thread
	0 bytes statically-allocated shared memory per block
	0 byte

In [ ]:
from model import run_exp
from model import default_params as params
import utils as utl
from brian2 import Hz

# config = {
#     'path_res'  : './results/example',                              # directory to store results
#     'path_comp' : './2023_03_23_completeness_630_final.csv',        # csv of the complete list of Flywire neurons
#     'path_con'  : './2023_03_23_connectivity_630_final.parquet',    # connectivity data
#     'n_proc'    : -1,                                               # number of CPU cores (-1: use all)
# }

config = {
    'path_res'  : './results/new_example',                              # directory to store results
    'path_comp' : './Completeness_783.csv',        # csv of the complete list of Flywire neurons
    'path_con'  : './Connectivity_783.parquet',    # connectivity data
    'n_proc'    : -1,                                               # number of CPU cores (-1: use all)
}

# Introduction
## Underlying connectivity data
The connectivity of the fly brain is stored in the folowing files:
- neurons present: `config['path_comp']`
- connectivity between neurons: `config['path_con]`

## Model parameters
The equation and constants for the leaky integrate and fire model are defined 
in the dictionary `default_params` in the beginning of the file `model.py`:

```
default_params = {
    # trials
    't_run'     : 1000 * ms,              # duration of trial
    'n_run'     : 30,                     # number of runs

    'v_0'       : -52 * mV,               # resting potential
    'v_rst'     : -52 * mV,               # reset potential after spike
    [...]
```
We can also change values
and pass the modified dictionary to the model (see Experiment 1).

## Addressing neurons
Here, we want to stimulate some sugar-sensing neurons in the right hemisphere.
The neurons of interest are defined via their flywire IDs:

In [2]:
neu_sugar = [
    720575940624963786,
    720575940630233916,
    720575940637568838,
    720575940638202345,
    720575940617000768,
    720575940630797113,
    720575940632889389,
    720575940621754367,
    720575940621502051,
    720575940640649691,
    720575940639332736,
    720575940616885538,
    720575940639198653,
    720575940620900446,
    720575940617937543,
    720575940632425919,
    720575940633143833,
    720575940612670570,
    720575940628853239,
    720575940629176663,
    720575940611875570,
]

For an easier identification, we define also a mapping from the flywire IDs to custom 
names. The above neurons are calles `sugar_1`, `sugar_2` etc:

In [3]:
flyid2name = { f: f'sugar_{i+1}' for i, f in enumerate(neu_sugar) }
flyid2name

{720575940624963786: 'sugar_1',
 720575940630233916: 'sugar_2',
 720575940637568838: 'sugar_3',
 720575940638202345: 'sugar_4',
 720575940617000768: 'sugar_5',
 720575940630797113: 'sugar_6',
 720575940632889389: 'sugar_7',
 720575940621754367: 'sugar_8',
 720575940621502051: 'sugar_9',
 720575940640649691: 'sugar_10',
 720575940639332736: 'sugar_11',
 720575940616885538: 'sugar_12',
 720575940639198653: 'sugar_13',
 720575940620900446: 'sugar_14',
 720575940617937543: 'sugar_15',
 720575940632425919: 'sugar_16',
 720575940633143833: 'sugar_17',
 720575940612670570: 'sugar_18',
 720575940628853239: 'sugar_19',
 720575940629176663: 'sugar_20',
 720575940611875570: 'sugar_21'}

# Running simulations
## Activating a set of neurons
To run a simulation exciting these nerons we have to call `run_exp` supplying the following:
- unique name for the simulation: `exp_name`
- a list of neurons we want to stimulate: `neu_sugar`
- the connectivity data: `config['path_comp']` and `config['path_con]`
- path to store the output: `config['path_res']`
- number of CPU cores use: `config['n_procs]`

Note that running this on Google Colab can take roughly 20 minutes; it is substantially faster on a local install, depending on the number of CPU cores. By default, the neurons are excited at 200 Hz.

In [4]:
# activate sugar sensing neurons
run_exp(exp_name='sugarR', neu_exc=neu_sugar, **config)

>>> Experiment:     sugarR
    Output file:    results/example/sugarR.parquet
    Excited neurons: 21
    Elapsed time:   131 s


The `.parquet` file created during a simulation contains all spikes events of all neurons in the model.
We load the data again from disk by passing a list of result files to the `utl.load_exps` function.

We can see from the size of the dataframe
that more than 400 000 spikes were generated by activating the sugar neurons (30 trials, 1 s each).

In [5]:
# load data from disk
df_spike = utl.load_exps([ './results/example/sugarR.parquet' ])
df_spike

,t,trial,flywire_id,exp_name
0,0.1815,0,720575940605513649,sugarR
1,0.4603,0,720575940605513649,sugarR
2,0.5665,0,720575940605513649,sugarR
3,0.6851,0,720575940605513649,sugarR
4,0.7804,0,720575940605513649,sugarR
...,...,...,...,...
406580,0.9006,29,720575940660229505,sugarR
406581,0.9181,29,720575940660229505,sugarR
406582,0.9343,29,720575940660229505,sugarR
406583,0.9630,29,720575940660229505,sugarR


The spike times can be converted to spike rates [Hz] via `utl.get_rate`, which requires the duration of each trial.
`utl.get_rate` returns `pandas.DataFrame` objects:
1. spike rate for each neuron (rows) in each experiment (column): `df_rate`
2. standard deviation of rate across trials: `df_rate_std`

For convenience, we can optionally pass the `flyid2name` dictionary to `utl.get_rate` in order to convert flywire IDs into
meaningful names.

We can see that only about 400 neurons show activity during the simulations.

In [6]:
# calculate spike rate and standard deviation
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
# sort by spike rate
df_rate.sort_values('sugarR', ascending=False)

exp_name,name,sugarR
flyid,,
720575940630797113,sugar_6,151.966667
720575940638202345,sugar_4,151.900000
720575940616885538,sugar_12,151.333333
720575940620900446,sugar_14,151.333333
720575940637568838,sugar_3,150.333333
...,...,...
720575940627739925,,0.033333
720575940644438551,,0.033333
720575940624816115,,0.033333


## Change stimulation frequency

We want to change the frequency of the stimulation of the sugar neurons.
To do so we modify the value for `r_poi` in the `default_params` dictionary and pass the altered dictionary to the `run_exp` function.

Note: Since physical quantities in `brian2` have to have the correct unit, we also need the `brian2.Hz` object 
to define a frequency.

In [7]:
# run with different frequency
params['r_poi'] = 100 * Hz

run_exp(exp_name='sugarR_100Hz', neu_exc=neu_sugar, params=params, **config)

>>> Experiment:     sugarR_100Hz
    Output file:    results/example/sugarR_100Hz.parquet
    Excited neurons: 21


WARNING    /root/anaconda3/envs/brain/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]


    Elapsed time:   98 s


We load the results via the `utl.load_exps` function and convert the spike events to rates with `utl.get_rate`

In [8]:
ps = [
    './results/example/sugarR.parquet',
    './results/example/sugarR_100Hz.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('sugarR_100Hz', ascending=False, inplace=True)
df_rate

exp_name,name,sugarR,sugarR_100Hz
flyid,,,
720575940622695448,,138.566667,114.333333
720575940629888530,,126.800000,102.466667
720575940616885538,sugar_12,151.333333,101.933333
720575940630233916,sugar_2,145.433333,101.433333
720575940627383685,,123.933333,100.933333
...,...,...,...
720575940637790527,,1.266667,NaN
720575940637902938,,0.266667,NaN
720575940643551392,,0.266667,NaN


## Silencing neurons
We want to silence the most active neurons individually to see how that changes the activity patterns.
We do so by passing the neuron IDs we want to silence as a list `run_exp` via the `neu_slnc` argument.
In the following example, we are silencing a single neuron `[ i ]` while exciting the sugar neurons `neu_sugar`. 
We can then investigate how silencing of each individual neuron affects the firing rate of a given neuron, say, MN9. 

In [9]:
#First, let's check on the MN9 firing rate when no neurons are silenced.
id_mn9 = 720575940660219265 #id for MN9
x = df_rate.loc[id_mn9, "sugarR_100Hz"]
print(f'Rate for neuron {id_mn9} is {x}')

Rate for neuron 720575940660219265 is 67.6


In [10]:
# IDs of 3 most active neurons. These neurons are all sugar-sensing neurons.
ids = df_rate.sort_values('sugarR_100Hz', ascending=False).index[:3]

for i in ids:
    run_exp(exp_name=f'sugarR-{i}', neu_exc=neu_sugar, neu_slnc=[ i ], params=params, **config)

>>> Experiment:     sugarR-720575940622695448
    Output file:    results/example/sugarR-720575940622695448.parquet
    Excited neurons: 21
    Silenced neurons: 1


WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 53886. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 53886. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 53886. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 53886. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 5388

    Elapsed time:   49 s
>>> Experiment:     sugarR-720575940629888530
    Output file:    results/example/sugarR-720575940629888530.parquet
    Excited neurons: 21
    Silenced neurons: 1


WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 89304. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 89304. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 89304. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 89304. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 8930

    Elapsed time:   56 s
>>> Experiment:     sugarR-720575940616885538
    Output file:    results/example/sugarR-720575940616885538.parquet
    Excited neurons: 21
    Silenced neurons: 1


WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 27293. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 27293. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 27293. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 27293. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'i' is an internal variable of group 'default_synapses', but also exists in the run namespace with the value 2729

    Elapsed time:   98 s


In [11]:
# output files
ps = [ f'./results/example/sugarR-{i}.parquet' for i in ids ]

# calculate spike rate and sort
df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'])
df_rate.loc[id_mn9, :].sort_values(ascending=True)

exp_name
sugarR-720575940616885538    65.400000
sugarR-720575940629888530    66.000000
sugarR-720575940622695448    70.566667
Name: 720575940660219265, dtype: float64